# Knapsack problem

## Intro

This is an integer linear programming problem in which the goal is to maximize the value transported in a knapsack limited by capacity by selecting the most suitable items.

## Model statement

$$
\begin{align}
    \text{max} \quad & \sum_{i \in I}{c_{i} x_{i}} \\
    \text{s.t.} \quad & \sum_{i \in I}{w_{i} x_{i}} \leq k_{w} & \forall ~i \in I\\
    & \sum_{i \in I}{v_{i} x_{i}} \leq k_{v} & \forall ~i \in I\\
    & x_{i} \in \left \{ 0, 1 \right \} & \forall ~i \in I\\
\end{align}
$$

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import pyomo.environ as pyo

## Using scipy

We must formulate this problem using the following matrix notation:

$$
\begin{align}
    \text{min} \quad & \boldsymbol{c}^T \boldsymbol{x} \\
    \text{s.t.} \quad & \boldsymbol{A}_{eq} \boldsymbol{x} = \boldsymbol{b}_{eq} \\
    & \boldsymbol{A}_{ub} \boldsymbol{x} \leq \boldsymbol{b}_{ub}\\
    & \boldsymbol{l} \leq \boldsymbol{x} \leq \boldsymbol{u}
\end{align}
$$

**Decision variables:**

$\boldsymbol{x}$ - Column vector with the amount of each item added to the knapsack

**Fixed parameters**

$\boldsymbol{c}$ - Cost associated with each decision variable (negative of its value)

$\boldsymbol{A}_{ub}$ - Matrix of inequality constraint terms\
$a_{1, i}$ Weight per unit of item $i\\$
$a_{2, i}$ Volume per unit of item $i$

$k_w$ - Knapsack weight capacity\
$k_v$ - Knapsack volume capacity

$b_{ub} = \begin{bmatrix} k_w \\ k_v \end{bmatrix}$

In [2]:
#Set of items
I = list(range(1, 11))

#Random seed
np.random.seed(12)

#Weight associated with each item
w = dict(zip(I, np.random.normal(loc=5.0, scale=1.0, size=10).clip(0.5, 10.0)))

#Volume associated with each item
v = dict(zip(I, np.random.normal(loc=6.0, scale=2.0, size=10).clip(0.5, 10.0)))

#Price associated with each item
price = dict(zip(I, np.random.normal(loc=10.0, scale=1.0, size=10).clip(0.5, 20.0)))

#knapsack capacity
kw, kv = 21.0, 22.0

In [3]:
#Costs
c = -np.array(list(price.values()))

#Inequality constraints matrix
A_ub = np.array([
    np.array(list(w.values())),
    np.array(list(v.values()))
])

#Upper bounds for linear inequality constraints
b_ub = np.array([kw, kv])

#Bounds (one quantity of each item)
bounds = [(0, 1),] * 10

In [4]:
sol = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

In [5]:
print(sol)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -44.81724486288822
              x: [ 1.000e+00  8.656e-01  0.000e+00  1.000e+00  0.000e+00
                   0.000e+00  0.000e+00  8.805e-01  1.000e+00  0.000e+00]
            nit: 2
          lower:  residual: [ 1.000e+00  8.656e-01  0.000e+00  1.000e+00
                              0.000e+00  0.000e+00  0.000e+00  8.805e-01
                              1.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  1.456e+00  0.000e+00
                              4.160e+00  5.104e-01  3.411e+00  0.000e+00
                              0.000e+00  5.386e+00]
          upper:  residual: [ 0.000e+00  1.344e-01  1.000e+00  0.000e+00
                              1.000e+00  1.000e+00  1.000e+00  1.195e-01
                              0.000e+00  1.000e+00]
                 marginals: [-1.181e+00  0.000e+00  0.000e+00 -4.314e+00
         

In [6]:
for i, item in enumerate(I):
    xi = sol.x[i]
    print(f"Item {item}: {xi:.3f}")

Item 1: 1.000
Item 2: 0.866
Item 3: 0.000
Item 4: 1.000
Item 5: 0.000
Item 6: 0.000
Item 7: 0.000
Item 8: 0.881
Item 9: 1.000
Item 10: 0.000


### Integrality constaints

Since version 1.9.0 scipy accepts integrality constraints as it now as a wrapper to a MILP solver.

$x_i \in \Z \space \forall i \in I$

In [7]:
integrality_vector = np.full(c.shape[0], 1)

sol_int = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=integrality_vector)

In [8]:
print(sol_int)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -39.88187183116921
              x: [ 1.000e+00  0.000e+00  1.000e+00  1.000e+00  0.000e+00
                   0.000e+00  0.000e+00  1.000e+00  0.000e+00  0.000e+00]
            nit: -1
          lower:  residual: [ 1.000e+00  0.000e+00  1.000e+00  1.000e+00
                              0.000e+00  0.000e+00  0.000e+00  1.000e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [ 0.000e+00  1.000e+00  0.000e+00  0.000e+00
                              1.000e+00  1.000e+00  1.000e+00  0.000e+00
                              1.000e+00  1.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
        

In [9]:
for i, item in enumerate(I):
    xi = sol_int.x[i]
    print(f"Item {item}: {xi:.3f}")

Item 1: 1.000
Item 2: 0.000
Item 3: 1.000
Item 4: 1.000
Item 5: 0.000
Item 6: 0.000
Item 7: 0.000
Item 8: 1.000
Item 9: 0.000
Item 10: 0.000


## Using pyomo

In [10]:
model = pyo.ConcreteModel()

### Sets

In [11]:
model.I = pyo.Set(initialize=I)

### Parameters

In [12]:
# Parameters of the knapsack
model.kw = pyo.Param(initialize=kw)
model.kv = pyo.Param(initialize=kv)

In [13]:
# Parameters of the items
model.v = pyo.Param(model.I, initialize=v)
model.w = pyo.Param(model.I, initialize=w)
model.c = pyo.Param(model.I, initialize=price)

### Variables

In [14]:
model.x = pyo.Var(model.I, within=pyo.Binary)

## Constraints

In [15]:
def weight_constraint(model):
    
    return sum(model.x[i] * model.w[i] for i in model.I) <= model.kw

model.weight_constraint = pyo.Constraint(rule=weight_constraint)

def volume_constraint(model):
    
    return sum(model.x[i] * model.v[i] for i in model.I) <= model.kv

model.volume_constraint = pyo.Constraint(rule=volume_constraint)

### Objective

In [16]:
def obj_function(model):
    
    return sum(model.x[i] * model.c[i] for i in model.I)
    
model.objective = pyo.Objective(rule=obj_function, sense=pyo.maximize)

In [17]:
opt = pyo.SolverFactory('glpk')
opt.options['tmlim'] = 120

In [18]:
solution = opt.solve(model)
print(solution)

    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'

In [ ]:
model.objective.display()

objective : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 39.88187183116921


In [ ]:
model.x.display()

x : Size=10, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Integers
      2 :     0 :   0.0 :     1 : False : False : Integers
      3 :     0 :   1.0 :     1 : False : False : Integers
      4 :     0 :   1.0 :     1 : False : False : Integers
      5 :     0 :   0.0 :     1 : False : False : Integers
      6 :     0 :   0.0 :     1 : False : False : Integers
      7 :     0 :   0.0 :     1 : False : False : Integers
      8 :     0 :   1.0 :     1 : False : False : Integers
      9 :     0 :   0.0 :     1 : False : False : Integers
     10 :     0 :   0.0 :     1 : False : False : Integers


In [ ]:
for i, item in enumerate(I):
    if abs(sol.x[i] - model.x[item].value) <= 1e-3:
        if sol.x[i] >= 1e-3:
            print(f"Item {item} was added in both situations")
        else:
            print(f"Item {item} was not added in any situation")
        
    elif sol.x[i] > model.x[item].value + 1e-3:
        if sol.x[i] == 1:
            print(f"Item {item} was completely added in relaxed problem only")
        else:
            xi = sol.x[i]
            print(f"Item {item} was partially added in relaxed problem only - value {xi:.2f}")
            
    elif sol.x[i] + 1e-3 < model.x[item].value:
        if sol.x[i] <= 1e-3:
            print(f"Item {item} was added only in integer problem")
        else:
            xi = sol.x[i]
            print(f"Item {item} was partially added in relaxed problem - value {xi:.2f} - but completely added in the integer version")

Item 1 was added in both situations
Item 2 was partially added in relaxed problem only - value 0.87
Item 3 was added only in integer problem
Item 4 was added in both situations
Item 5 was not added in any situation
Item 6 was not added in any situation
Item 7 was not added in any situation
Item 8 was partially added in relaxed problem - value 0.88 - but completely added in the integer version
Item 9 was partially added in relaxed problem only - value 1.00
Item 10 was not added in any situation


In [ ]:
model.weight_constraint.display()

weight_constraint : Size=1
    Key  : Lower : Body               : Upper
    None :  None : 18.894462023985934 :  21.0


In [ ]:
model.volume_constraint.display()

volume_constraint : Size=1
    Key  : Lower : Body              : Upper
    None :  None : 20.73819050160827 :  22.0
